<a href="https://colab.research.google.com/github/VimalRency/Spectogram/blob/main/classification_of_spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
from google.colab import files
uploaded = files.upload()

# Move uploaded files to the specified directory
import os
import shutil

upload_dir = '/content/sample_data/spectrogram'
os.makedirs(upload_dir, exist_ok=True)

for filename, content in uploaded.items():
    with open(os.path.join(upload_dir, filename), 'wb') as f:
        f.write(content)


Saving ano.png to ano.png
Saving cno.png to cno.png
Saving gno.png to gno.png
Saving kno.png to kno.png
Saving vno.png to vno.png


In [9]:
from google.colab import files
uploaded = files.upload()

# Move uploaded files to the specified directory
import os
import shutil

upload_dir = '/content/sample_data/spectrogram'
os.makedirs(upload_dir, exist_ok=True)

for filename, content in uploaded.items():
    with open(os.path.join(upload_dir, filename), 'wb') as f:
        f.write(content)


Saving aish.png to aish.png
Saving carmel.png to carmel.png
Saving gayathri.png to gayathri.png
Saving kathy.png to kathy.png
Saving vimal.png to vimal.png


In [22]:
import numpy as np
import tensorflow as tf
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Define data directory and parameters
data_dir = '/content/sample_data/spectrogram'
img_size = (128, 128)

# Function to extract features using VGG16
def extract_features(data_dir, img_size=(128, 128)):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

    features = []
    labels = []

    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.png'):
                img_path = os.path.join(root, file)
                img = load_img(img_path, target_size=img_size)
                img = img_to_array(img)
                img = np.expand_dims(img, axis=0)
                img = tf.keras.applications.vgg16.preprocess_input(img)

                feature = base_model.predict(img)
                feature = np.ravel(feature)  # Flatten feature vector

                features.append(feature)
                labels.append(os.path.basename(root))  # Use directory name as label

    features = np.array(features)
    labels = np.array(labels)

    return features, labels, base_model

# Extract features and labels
X, y, base_model = extract_features(data_dir, img_size=img_size)

# Encode class labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


1/1 [==============================] - 0s 358ms/step


In [23]:
# Train SVM
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)


SVC(kernel='linear', random_state=42)

In [24]:
# Evaluate SVM
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

# Optionally, print classification report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Accuracy: 0.00%
              precision    recall  f1-score   support

          no       0.00      0.00      0.00       1.0
 spectrogram       0.00      0.00      0.00       3.0
         yes       0.00      0.00      0.00       1.0

    accuracy                           0.00       5.0
   macro avg       0.00      0.00      0.00       5.0
weighted avg       0.00      0.00      0.00       5.0



In [25]:
def predict_spectrogram(model, img_path, label_encoder, base_model, img_size=(128, 128)):
    img = load_img(img_path, target_size=img_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)

    feature = base_model.predict(img)
    feature = np.ravel(feature)  # Flatten feature vector

    predicted_class_idx = model.predict([feature])[0]
    predicted_class = label_encoder.inverse_transform([predicted_class_idx])[0]

    return predicted_class

# Example usage
new_image_path = '/content/sample_data/spectrogram/new_image.png'  # Replace with your new image path

# Check if the file exists
if os.path.exists(new_image_path):
    predicted_class = predict_spectrogram(svm_model, new_image_path, label_encoder, base_model)
    print(f'Predicted Class: {predicted_class}')
else:
    print(f'Error: Image file "{new_image_path}" not found.')


1/1 [==============================] - 0s 189ms/step
Predicted Class: spectrogram
